In [ ]:
import torch
import os
import sys
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)

import Double_Pendulum.Learning.autoencoders as autoencoders
import Double_Pendulum.robot_parameters as robot_parameters
import Double_Pendulum.dynamics as dynamics
import Double_Pendulum.transforms as transforms

import Double_Pendulum.normal_form as normal_form

from datetime import datetime

rp = robot_parameters.LUMPED_PARAMETERS
rp["m0"] = 0.
print(rp)
model = autoencoders.Analytic_transformer(rp)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_cw = False

%load_ext autoreload
%autoreload 2

In [ ]:
def plot_single_vs_time(single_list, dt, ylabel, title, ylim = None, log = False):
    time = torch.linspace(0, (single_list.shape[0] - 1) * dt, single_list.shape[0]).numpy()
    single_np = single_list.cpu().numpy()

    plt.figure(figsize=(7, 3))
    plt.plot(time, single_np, label=ylabel)
    plt.xlabel("Time (s)")
    plt.ylabel(ylabel)
    plt.title(title)
    if ylim is not None:
        plt.ylim(ylim)
    if log:
        plt.yscale("log")
    plt.legend()
    plt.grid(True)
    #plt.yscale("log")
    plt.tight_layout()
    plt.show()

def plot_double_vs_time(double_list, dt, ylabel, title, ylim = None, log = False):
    time = torch.linspace(0, (double_list.shape[0] - 1) * dt, double_list.shape[0]).numpy()
    double_np = double_list.cpu().numpy()

    plt.figure(figsize=(7, 3))
    plt.plot(time, double_np[:, 0], label=ylabel+"0")
    plt.plot(time, double_np[:, 1], label=ylabel+"1")
    plt.xlabel("Time (s)")
    plt.ylabel(ylabel)
    plt.title(title)
    if ylim is not None:
        plt.ylim(ylim)
    if log:
        plt.yscale("log")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_quad_vs_time(quad_list, dt, ylabel, title, ylim = None, log = False):
    time = torch.linspace(0, (quad_list.shape[0] - 1) * dt, quad_list.shape[0]).numpy()
    quad_np = quad_list.cpu().numpy()
    

    plt.figure(figsize=(7, 3))
    plt.plot(time, quad_np[:, 0], label="Y")
    plt.plot(time, quad_np[:, 1], label="Y'")
    plt.plot(time, quad_np[:, 2], label="Y''")
    plt.plot(time, quad_np[:, 3], label="Y'''")
    plt.xlabel("Time (s)")
    plt.ylabel(ylabel)
    plt.title(title)
    if ylim is not None:
        plt.ylim(ylim)
    if log:
        plt.yscale("log")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
K = torch.tensor([[1., 10., 10., 10.]]).to(device)
k_spring = 0.5
if normal_form.check_stable_gains(K):
    print("Gains are stable")

#### Define desired conditions

In [ ]:
# THE FOLLOWING COORDINATES RESULT IN STABLE Y_DES FOR k_spring = 0.5
xy_des_real = torch.tensor([1.9218631, -2.5156569]).requires_grad_().to(device)
q_d_des = torch.tensor([[0., 0.]]).requires_grad_().to(device)

q_des = transforms.inverse_kinematics(xy_des_real, rp, is_clockwise=model_cw).unsqueeze(0)
is_clockwise_des = transforms.check_clockwise(q_des.squeeze(0))

th_des = model.encoder_vmap(q_des)
th_d_des = (model.jacobian_enc(q_des) @ q_d_des.T).T

# Use this if you want to directly define a desired th_des
#th_des = torch.tensor([[7.5160644,  1.5604]]).requires_grad_().to(device)
#th_d_des = torch.tensor([[0.,  0.]]).requires_grad_().to(device)



print("xy_des:", xy_des_real)
print("q_des:", q_des)
print("th_des", th_des)

#### Calculate desired dynamics

In [ ]:
J_h_inv_des = model.jacobian_dec(th_des, is_clockwise_des).squeeze(0)
J_h_inv_trans_des = torch.transpose(J_h_inv_des, 0, 1)

q_hat_des = model.decoder_vmap(th_des, is_clockwise_des)
q_d_hat_des = (model.jacobian_dec(th_des, clockwise=is_clockwise_des) @ th_d_des.T).T
#xy_des_est, _ = transforms.forward_kinematics(rp, q_hat_des[0])

M_q_des, C_q_des, G_q_des = dynamics.dynamical_matrices(rp, q_hat_des.squeeze(0), q_d_hat_des.squeeze(0))
G_q_des = dynamics.add_spring_force_G_q(rp, q_hat_des, G_q_des, k_spring)
A_q_des = dynamics.input_matrix(rp, q_des.squeeze(0))

_, _, G_th_des = transforms.transform_dynamical_from_inverse(M_q_des, C_q_des, G_q_des, th_des, th_d_des, J_h_inv_des, J_h_inv_trans_des)
M_th_des = torch.tensor([[rp["m1"], 0.], [0., rp["m1"]]]).to(device).requires_grad_(True)
M_th_des = M_th_des * th_des/th_des
A_th_des = transforms.transform_input_matrix_from_inverse_trans(A_q_des, J_h_inv_trans_des)

Y_des_u = normal_form.calculate_Y(th_des, th_d_des, M_th_des, G_th_des, device)
Y_des = torch.tensor([[Y_des_u[0,0]], [0], [0], [0]]).to(device)

print("M_th des:\n", M_th_des)
print("G_th des:\n", G_th_des)

print("Y_des_u:\n", Y_des_u)
print("Y_des:\n", Y_des)

#### Define start conditions

In [ ]:
#xy_start = torch.tensor([2, -1.9]).requires_grad_().to(device)
xy_start = xy_des_real * 1.001
q_d_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)
q_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)

q_start = transforms.inverse_kinematics(xy_start, rp, is_clockwise=model_cw).unsqueeze(0)
is_clockwise_start = transforms.check_clockwise(q_start.squeeze(0))

th_start = model.encoder_vmap(q_start)
th_d_start = (model.jacobian_enc(q_start) @ q_d_start.T).T

print("xy_start:", xy_start)
print("q_start:", q_start)
print("th_start:", th_start)

#### Calculate starting dynamics

In [ ]:
J_h_inv_start = model.jacobian_dec(th_start, is_clockwise_start).squeeze(0)
J_h_inv_trans_start = torch.transpose(J_h_inv_start, 0, 1)

q_hat_start = model.decoder_vmap(th_start, is_clockwise_start)
q_d_hat_start = (model.jacobian_dec(th_start, clockwise=is_clockwise_start) @ th_d_start.T).T

M_q_start, C_q_start, G_q_start = dynamics.dynamical_matrices(rp, q_hat_start.squeeze(0), q_d_hat_start.squeeze(0))
G_q_start = dynamics.add_spring_force_G_q(rp, q_hat_start, G_q_start, k_spring)
A_q_start = dynamics.input_matrix(rp, q_hat_start.squeeze(0))

_, _, G_th_start = transforms.transform_dynamical_from_inverse(M_q_start, C_q_start, G_q_start, th_start, th_d_start, J_h_inv_start, J_h_inv_trans_start)
M_th_start = torch.tensor([[rp["m1"], 0.], [0., rp["m1"]]]).to(device).requires_grad_(True)
M_th_start = M_th_start * th_start/th_start
A_th_start = transforms.transform_input_matrix_from_inverse_trans(A_q_start, J_h_inv_trans_start)

Y_start = normal_form.calculate_Y(th_start, th_d_start, M_th_start, G_th_start, device)
v_start = normal_form.calculate_v(Y_start, Y_des, K)
alpha_start, beta_start = normal_form.calculate_alpha_beta(th_start, th_d_start, M_th_start, G_th_start, A_th_start, Y_start)
u_start = normal_form.calculate_u(alpha_start, beta_start, v_start)
y_iv_start = normal_form.calculate_y_iv(alpha_start, beta_start, u_start)

print("v_start:", v_start)
print("u_start:", u_start)
print("yiv_start:", y_iv_start)

print("M_th start:\n", M_th_start)
print("G_th start:\n", G_th_start)
print("A_th start:\n", A_th_start)

print("Y_start:", Y_start)

### Sim in $\theta$-space

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

th_series_thsim, th_d_series_thsim, th_dd_series_thsim = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
q_series_thsim, q_d_series_thsim, q_dd_series_thsim = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
Y_series_thsim = torch.empty((0,4)).to(device)
v_series_thsim = torch.empty((0,1)).to(device)
u_series_thsim = torch.empty((0,1)).to(device)
alpha_beta_series_thsim = torch.empty((0,2)).to(device)


dt = 0.01
t_end = 5
t_series = torch.arange(0, t_end, dt)

th = th_start
th_d = th_d_start

is_clockwise = transforms.check_clockwise(q_start.squeeze(0))

for t in torch.arange(0, t_end, dt):
    t_string = "Time: [" + str(t.item().__round__(3)) + "/" + str(t_end) + ".0]"
    print(t_string, end='\r', flush=True)

    #print("th:", th)
    #print("th_d:", th_d)

    q_hat = model.decoder_vmap(th, clockwise=model_cw)
    q_d_hat = (model.jacobian_dec(th) @ th_d.T).T
    
    """ Obtain Jacobian, dynamical matrices"""
    
    J_h_inv = model.jacobian_dec(th, is_clockwise).squeeze(0)
    J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)

    M_q_est, C_q_est, G_q_est = dynamics.dynamical_matrices(rp, q_hat.squeeze(0), q_d_hat.squeeze(0))
    G_q_est = dynamics.add_spring_force_G_q(rp, q_hat, G_q_est, k_spring)
    A_q_est = dynamics.input_matrix(rp, q_hat.squeeze(0))


    """ Feed-forward simulation of the system, not on real dynamics """
    _, _, G_th = transforms.transform_dynamical_from_inverse(M_q_est, C_q_est, G_q_est, th, th_d, J_h_inv, J_h_inv_trans)
    M_th = torch.tensor([[rp["m1"], 0.], [0., rp["m1"]]]).to(device).requires_grad_(True)
    M_th = M_th * th/th
    A_th = transforms.transform_input_matrix_from_inverse_trans(A_q_est, J_h_inv_trans)

    Y = normal_form.calculate_Y(th, th_d, M_th, G_th, device)
    alpha, beta = normal_form.calculate_alpha_beta(th, th_d, M_th, G_th, A_th, Y)



    #print("alpha:", alpha)
    #print("beta:", beta)
    v = normal_form.calculate_v(Y, Y_des, K)
    u = normal_form.calculate_u(alpha, beta, v)
    #print("Y:", Y)

    #print("u:", u)
    if t == 0:
        print("t: t")
        print("th:", th)
        print("Y:", Y)
        #print("Y_des:", Y_des)
        print("v:", v.item())
        print("u:", u.item())
        print("\n")

    """ Recalculate "real system" based on modified matrices in theta-space. """



    """ Update the real system and apply latent control input. """

    

    tau_th = A_th * u
    th_dd = (torch.pinverse(M_th) @ (tau_th - G_th)).T
    th_d = th_d + th_dd * dt
    th = th + th_d * dt

    if th[0,0] < 0:
        th[0,0] = -th[0,0]
        th[0,1] += torch.pi
    
    if th[0,1] > torch.pi:
        th[0,1] -= 2*torch.pi
    if th[0,1] < -torch.pi:
        th[0,1] += 2*torch.pi

    q_est = model.decoder_vmap(th, clockwise=is_clockwise)
    q_est = transforms.wrap_to_pi(q_est)
    q_d_est = (model.jacobian_dec(th, clockwise=is_clockwise) @ th_d.T).T




    """ Store data for plotting """

    th_series_thsim = torch.cat((th_series_thsim, th.detach()), dim=0)
    th_d_series_thsim = torch.cat((th_d_series_thsim, th_d.detach()), dim=0)
    th_dd_series_thsim = torch.cat((th_dd_series_thsim, th_dd.detach()), dim=0)

    q_series_thsim = torch.cat((q_series_thsim, q_est.detach()), dim=0)
    q_d_series_thsim = torch.cat((q_d_series_thsim, q_d_est.detach()), dim=0)

    v_series_thsim = torch.cat((v_series_thsim, v.detach()), dim=0)
    u_series_thsim = torch.cat((u_series_thsim, u.detach()), dim=0)
    Y_series_thsim = torch.cat((Y_series_thsim, Y.detach().T))
    alpha_beta_series_thsim = torch.cat((alpha_beta_series_thsim, torch.tensor([[alpha, beta]]).to(device)))
    #print("")

    if torch.isnan(th[0,0]):
        break

    
    


### Sim in Y-space

In [ ]:
Y_series_Ysim = torch.empty((0,4)).to(device)
v_series_Ysim = torch.empty((0,1)).to(device)

Y = normal_form.calculate_Y(th_start, th_d_start, M_th_start, G_th_start, device)
#print(Y)
t_end = 10.
dt = 0.01

Y_A = torch.tensor([[0., 1., 0., 0.],
                    [0., 0., 1., 0.],
                    [0., 0., 0., 1.],
                    [0., 0., 0., 0.]]).to(device)

Y_B = torch.tensor([[0.], [0.], [0.], [1.]]).to(device)



for t in torch.arange(0, t_end, dt):
    v = normal_form.calculate_v(Y, Y_des, K)

    Y_series_Ysim = torch.cat((Y_series_Ysim, Y.detach().T))

    Y_dot = Y_A @ Y + Y_B * v
    Y = Y + Y_dot * dt
    
    v_series_Ysim = torch.cat((v_series_Ysim, v.detach()))



In [ ]:
show_thsim = True
show_Ysim = True

if show_thsim:
    plot_quad_vs_time(Y_series_thsim[:50], dt, "Y_thsim", "Y_thsim vs time", ylim=(-3, 3))
    plot_single_vs_time(v_series_thsim[:50], dt, "v_thsim", "v_thsim vs time", ylim=(-10, 10))
    #plot_single_vs_time(u_series_thsim[:50], dt, "u_thsim", "u_thsim vs time")
    plot_double_vs_time(th_series_thsim[:50], dt, "th_thsim", "th_thsim vs time", (-2, 10))
    #plot_double_vs_time(q_series_thsim[:50], dt, "q_thsim", "q_thsim vs time")

if show_Ysim:
    plot_quad_vs_time(Y_series_Ysim[:50], dt, "Y_Ysim", "Y_Ysim vs time", ylim=(-3, 3))
    plot_single_vs_time(v_series_Ysim[:50], dt, "v_Ysim", "v_Ysim vs time", ylim=(-10, 10))



#### Calculate Y as a function of $\theta_0$

In [ ]:
plot_th_min, plot_th_max = -5, 15
th0_range = torch.linspace(plot_th_min, plot_th_max, 200).unsqueeze(1).to(device)
th1_range = (torch.ones(th0_range.size(0)).to(device) * th_des[0, 1]).unsqueeze(1)
th_des_range = torch.cat((th0_range, th1_range), dim=1)

y_plot_list = torch.empty(4,0).to(device)

for th_des_plot in th_des_range:
    is_clockwise_des_plot = False
    
    th_des_plot = th_des_plot.unsqueeze(0)
    q_hat_des_plot = model.decoder_vmap(th_des_plot, is_clockwise_des_plot)
    q_d_hat_des_plot = (model.jacobian_dec(th_des_plot, clockwise=is_clockwise_des_plot) @ th_d_des.T).T

    J_h_inv_des = model.jacobian_dec(th_des_plot, is_clockwise_des_plot).squeeze(0)
    J_h_inv_trans_des = torch.transpose(J_h_inv_des, 0, 1)

    M_q_des, C_q_des, G_q_des = dynamics.dynamical_matrices(rp, q_hat_des_plot.squeeze(0), q_d_hat_des_plot.squeeze(0))
    G_q_des = dynamics.add_spring_force_G_q(rp, q_hat_des_plot, G_q_des, k_spring)
    G_th_des = J_h_inv_trans_des @ G_q_des
    #_, _, G_th_des = transforms.transform_dynamical_from_inverse(M_q_des, C_q_des, G_q_des, th_des_plot, th_d_des, J_h_inv_des, J_h_inv_trans_des)
    M_th_des = torch.tensor([[rp["m1"], 0.], [0., rp["m1"]]]).to(device).requires_grad_(True)
    M_th_des = M_th_des * th_des_plot/th_des_plot	
    
    
    Y_des = normal_form.calculate_Y(th_des_plot, th_d_des, M_th_des, G_th_des, device)
    y_plot_list = torch.cat((y_plot_list, Y_des), dim = -1)



In [ ]:
import math
from sympy import symbols, Eq, solve

# Circle parameters
r = rp["l0"] + rp["l1"]

# Points on the line
x1, y1 = xy_des_real[0].item(), xy_des_real[1].item()# 1.9219, -2.5157
x2, y2 = rp["xa"], rp["ya"]

# Line direction vector
dx = x2 - x1
dy = y2 - y1

# Parametric line: x = x1 + t*dx, y = y1 + t*dy
t = symbols('t')
x = x1 + t * dx
y = y1 + t * dy

# Equation of the circle: x^2 + y^2 = r^2
circle_eq = Eq(x**2 + y**2, r**2)

# Solve for t
solutions = solve(circle_eq, t)

# Find the coordinates of the intersection points
intersection_points = [(x1 + float(sol) * dx, y1 + float(sol) * dy) for sol in solutions]

# Calculate angle of the line with respect to horizontal in radians
angle_radians = math.atan2(dy, dx)

# Compute distances from point A to each intersection point
distances = [math.hypot(x2 - px, y2 - py) for px, py in intersection_points]

for i, distance in enumerate(distances):
    distances[i] = round(distance, 4)

short_distance = min(distances)
long_distance = max(distances)

rounded_points = []
for intersection in intersection_points:
    intx = round(intersection[0], 4)
    inty = round(intersection[1], 4)
    rounded_points.append((intx, inty))
    

#print("Intersection points th1 and circle:", rounded_points)
#print("Angle (th1) in radians:", round(angle_radians, 4))
#print("Distances between intersection points and actuator point:\n", distances)


#### Plot the resulting Y as a function of $\theta_0$

In [ ]:

for i in range(4):
    plt.plot(th0_range.squeeze(0).cpu().numpy(), y_plot_list[i].cpu().numpy())
    plt.hlines(0, plot_th_min, plot_th_max, colors="k", linestyles="--")
    vline_min = torch.min(torch.min((y_plot_list[i]), torch.tensor(-2))).cpu().numpy()
    vline_max = torch.max(torch.max((y_plot_list[i]), torch.tensor(2))).cpu().numpy()
    plt.vlines(0, vline_min, vline_max, colors="k", linestyles="--")
    plt.vlines(short_distance, vline_min, vline_max, colors="r", linestyles="--")
    plt.vlines(long_distance, vline_min, vline_max, colors="r", linestyles="--")
    
    plt.title("Y[" + str(i) + "] vs th0, for constant th1")
    plt.xlim(-2.5, 12.5)
    if i == 2:
        plt.ylim(-1, 1)
    plt.show()

#### Two sets of ($\theta_0$, $\theta_1$, $\dot{\theta}_0$, $\dot{\theta}_1$) result in the same Y:

In [ ]:
th_des_item_set = (torch.tensor([[7.5160644,  1.5604]]), torch.tensor([[2.525,  1.5604]]))
for th_des_item in th_des_item_set:

    # Use this if you want to directly define a desired th_des
    th_des = th_des_item.requires_grad_().to(device)
    th_d_des = torch.tensor([[0.,  0.]]).requires_grad_().to(device)

    is_clockwise_des = False

    

    J_h_inv_des = model.jacobian_dec(th_des, is_clockwise_des).squeeze(0)
    J_h_inv_trans_des = torch.transpose(J_h_inv_des, 0, 1)

    q_hat_des = model.decoder_vmap(th_des, is_clockwise_des)
    q_d_hat_des = (model.jacobian_dec(th_des, clockwise=is_clockwise_des) @ th_d_des.T).T
    #xy_des_est, _ = transforms.forward_kinematics(rp, q_hat_des[0])

    M_q_des, C_q_des, G_q_des = dynamics.dynamical_matrices(rp, q_hat_des.squeeze(0), q_d_hat_des.squeeze(0))
    print("G_q_des:", G_q_des)
    G_q_des = dynamics.add_spring_force_G_q(rp, q_hat_des, G_q_des, k_spring)

    _, _, G_th_des = transforms.transform_dynamical_from_inverse(M_q_des, C_q_des, G_q_des, th_des, th_d_des, J_h_inv_des, J_h_inv_trans_des)
    M_th_des = torch.tensor([[rp["m1"], 0.], [0., rp["m1"]]]).to(device).requires_grad_(True)
    M_th_des = M_th_des * th_des/th_des

    Y_des_u = normal_form.calculate_Y(th_des, th_d_des, M_th_des, G_th_des, device)
    Y_des = torch.tensor([[Y_des_u[0,0]], [0], [0], [0]]).to(device)

    print("th_des", th_des)
    print("Y_des_u:\n", Y_des_u)
    print("\n")
    